# Gather MITRE ATT&CK detection guidance for techniques
### Input: line-separated text file with list of MITRE ATT&CK technique IDs
### Output: Excel file technique name, number and detection guidance


### Import required libraries. Define filepath.

In [1]:
from attackcti import attack_client
import json
import numpy
import pandas as pd
import numpy as np
import xlsxwriter

inputFile = '/Users/dan.kaiser/OneDrive - LogRhythm, Inc/Documents/ransomware_module/techniques_to_detect' #Enter the full path and file name of the text file containing the MITRE Software items you would like analyzed
outputFile = '/Users/dan.kaiser/OneDrive - LogRhythm, Inc/Documents/ransomware_module/detection_guidance.xlsx' ##Enter the full path and file name of the Excel (xlsx) file that you would like as output of this script

### Load the MITRE ATT&CK Enterprise framework

In [2]:
lift = attack_client()
%time all_enterprise = lift.get_all_enterprise()

CPU times: user 6.75 s, sys: 343 ms, total: 7.1 s
Wall time: 13 s


### Read list of techniques from a file

In [3]:
techniques = []
techniques = [line.rstrip() for line in open(inputFile)]

In [4]:
for technique in techniques:
    print(technique)

T1486
T1490
T1059.003
T1562.001
T1489
T1106
T1083
T1027
T1082
T1140
T1070.004
T1112
T1059.001
T1016
T1047
T1057
T1547.001
T1573.002
T1055.001
T1105


### Generate a list of techniques for each software

In [7]:
techniqueAnalysis = []
for technique in techniques:
    for t in all_enterprise['techniques']:
        techniqueListItem = {}
        if t['external_references'][0]['external_id'] == technique:
            techniqueListItem['techniqueNumber'] = technique
            techniqueListItem['techniqueName'] = t['name']
            techniqueListItem['DetectionGuidance'] = t['x_mitre_detection']
            techniqueAnalysis.append(techniqueListItem)
print(techniqueAnalysis)

[{'techniqueNumber': 'T1486', 'techniqueName': 'Data Encrypted for Impact', 'DetectionGuidance': 'Use process monitoring to monitor the execution and command line parameters of binaries involved in data destruction activity, such as vssadmin, wbadmin, and bcdedit. Monitor for the creation of suspicious files as well as unusual file modification activity. In particular, look for large quantities of file modifications in user directories.\n\nIn some cases, monitoring for unusual kernel driver installation activity can aid in detection.\n\nIn cloud environments, monitor for events that indicate storage objects have been anomalously replaced by copies.'}, {'techniqueNumber': 'T1490', 'techniqueName': 'Inhibit System Recovery', 'DetectionGuidance': 'Use process monitoring to monitor the execution and command line parameters of binaries involved in inhibiting system recovery, such as vssadmin, wbadmin, and bcdedit. The Windows event logs, ex. Event ID 524 indicating a system catalog was dele

### Load the techique list into a dataframe

In [8]:
techniques = pd.DataFrame(techniqueAnalysis)

### Confirm the dataframe is populated by reviewing per-tactic counts

In [9]:
techniques['techniqueNumber'].value_counts()


T1140        1
T1489        1
T1547.001    1
T1016        1
T1083        1
T1059.001    1
T1082        1
T1105        1
T1112        1
T1055.001    1
T1486        1
T1059.003    1
T1047        1
T1573.002    1
T1027        1
T1106        1
T1562.001    1
T1057        1
T1070.004    1
T1490        1
Name: techniqueNumber, dtype: int64

### Write out the techniques dataframes into an Excel workbook

In [10]:
with pd.ExcelWriter(outputFile) as writer:
    techniques.to_excel(writer, sheet_name='Techniques')
